<a href="https://colab.research.google.com/github/lordbigot/UTS_ML2019_ID13191655/blob/master/A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider a machine with three arms. When the first arm is pulled, the machine returns \$5. When the second arm is pulled, the machine returns nothing. When the third arm is pulled, the machine has a 10% chance of returning \$1000 dollars. If the arms are pulled indefinitely, the third arm will produce the greatest reward. However, an observer can only learn the nature of this machine by pulling its arms. To get a complete picture, they are likely to start by pulling each arm once. In this case, there is a 90% chance that they will recieve no money from the third arm. When this occurs, they cannot distinguish between the second and third arms. From their perspective, it is possible neither arm will produce any reward.

This thought experiment brings me to a preliminary algorithm. First, each arm is pulled once. Then, the arms are sorted by their overall output. After this point, for as long as the machine can continue running, the arm with the greatest average output is pulled until the number of times it has been pulled exceeds n^2, where n is the number of times the arm with the lowest average output has been pulled. Each other arm is pulled, in series of highest to lowest average, until the number of times it has been pulled exceeds n. This algorithm will quickly scale, pulling the best-performing lever frequently, but continuing to pull the other levers as the risk persists that an otherwise poor-performing lever is the optimal solution.

Consider a machine with 12 levers. You will only be able to pull a lever a number of times between 2 and 12. You are unaware of how many pulls you will recieve. Assume, each lever returns a constant reward, but the expected reward is unknown to you. In this case, it is apparent that attempting to pull as many unknown levers as possible will produce middling results, as will pulling one lever as many times as possible. An optimal solution may be to pull k levers, and then pull the lever that produced the highest reward. For all number of pulls n between 2 and 12, the ideal number for k sits between 2 and (n+1)/2 inclusive. If the distribution of lever rewards across the machine has a strong left skew, pulling the lever twice is ideal, as it is where the lowest values of lever are least commonly pulled. This is because there are the least chances of testing a low-valued lever, but if one is found, another lever can be used. Keep in mind that the algorithm has no way of evaluating if the reward is below normal until a second lever is pulled. If the distribution has a strong right skew, pulling the lever (n+1)/2 times is ideal, as it is where the highest values are most commonly pulled. This is because the best product from splitting a number into two numbers and then multiplying them together is a square. If n = 8, and k = 5, there are 5 chances of pulling the best lever, and if it is found, it will be pulled 3 more times, for a total of 4 times, and a product of 20 times across all possible cases. If n = 8, and k = 7, there are 7 chances of pulling the best lever. However, if it is found, it will be only be pulled one more time, for a total of 2 times. This produces the comparably poor product of 14 pulls across all possible cases. An algorithm to identify the proportions of possible pulls is included below.

In [0]:
def list_permutations(n, target_index, current_index):
    if current_index == 0:
        new_list = []
        for i in range(n):
            new_list.append([i])
        return new_list
    list_below = list_permutations(n, target_index, current_index - 1)
    new_list = []
    for row in list_below:
        new_elements = []
        for i in range(n):
            found_element = False
            for element in row:
                found_element = found_element or element == i
            if not found_element:
                new_elements.append([i])
        for element in new_elements:
            new_list.append(row + element)
    return new_list


def complete_list(n, data):
    missing_rows = n - len(data[0])
    for row in data:
        maximum_class = 0
        for element in row:
            maximum_class = max(maximum_class, element)
        for i in range(missing_rows):
            row.append(maximum_class)
    return data


def frequency_analysis(n, data):
    new_list = []
    for i in range(n):
        new_list.append(0)
    for row in data:
        for element in row:
            new_list[element] += 1
    divisor = len(data) * n
    for i in range(n):
        new_list[i] /= divisor
    return new_list


def frequency_comparison(n):
    print("n:", n)
    for i in range(n):
        print("k:", i+1)
        print(frequency_analysis(n, complete_list(n, list_permutations(n, i, i)))

Referring to our earlier problem, for all cases 3 <= n <= 12; k = 2 is a good solution. However, for the majority of values, k = 3 is likely to be a more rounded solution, less dependent on the assumption of a left skew. In this thought experiment, nothing is gained from pulling an apparently good lever before a new one is discovered. However, this will not always be the case. If we now assume that each lever has a variable result, then it may be useful to gather additional information on apparently good levers before moving onto other levers. In the short-term, this may increase overall output, and in the long-term, it will be insignificant, justified by the increased certainty of the effect of the lever. 

After examining this experiment, I am drawn to amend my earlier algorithm. This algorithm now has two processes. The overall process will pull two new levers, call the subprocess three times, pull a new lever, call the subprocess three times, pull another lever, and so on until it runs out new levers, at which point it continually calls the subprocess. The subprocess, when called, will sort the known levers from greatest average output to lowest average output. It will pull the first lever if the number of times it has been pulled is less than (x+1)^2, where x is the number of times the last lever has been pulled. Otherwise, it will pull the next following lever that has been pulled less than x+1 times. This algorithm will effectively produce the same results as the last algorithm if every lever can be pulled, but will produce above-average results faster than previously possible. This algorithm is encoded below.

In [0]:
def partition_levers(levers, output_sums, output_instances, low, high):
    i = (low - 1)
    pivot = output_sums[high] / output_instances[high]
    for j in range(low, high):
        if output_sums[j] / output_instances[j] >= pivot:
            i += 1
            levers[i], levers[j] = levers[j], levers[i]
            output_sums[i], output_sums[j] = output_sums[j], output_sums[i]
            output_instances[i], output_instances[j] = output_instances[j], output_instances[i]
    levers[i+1], levers[high] = levers[high], levers[i+1]
    output_sums[i+1], output_sums[high] = output_sums[high], output_sums[i+1]
    output_instances[i+1], output_instances[high] = output_instances[high], output_instances[i+1]
    return (i+1)


def quick_lever_sort(levers, output_sums, output_instances, low, high):
    if low < high:
        pi = partition_levers(levers, output_sums, output_instances, low, high)
        quick_lever_sort(levers, output_sums, output_instances, low, pi-1)
        quick_lever_sort(levers, output_sums, output_instances, pi+1, high)


def pull_known_lever(levers, output_sums, output_instances):
    quick_lever_sort(levers, output_sums, output_instances, 0, len(output_sums) - 1)
    x = output_instances[len(output_instances) - 1]
    if output_instances[0] < pow(x + 1, 2):
        output_sums[0] += pull_lever(levers[0])
        output_instances[0] += 1
        return
    current_lever = 1
    while current_lever < len(output_sums):
        if output_instances[current_lever] < x + 1:
            output_sums[current_lever] += pull_lever(levers[current_lever])
            output_instances[current_lever] += 1
            return
        current_lever += 1


def algorithm(levers):
    output_sums = []
    output_instances = []
    output_sums.append(pull_lever(levers[0]))
    output_instances.append(1)
    current_lever = 1
    while current_lever < len(levers):
        output_sums.append(pull_lever(levers[current_lever]))
        output_instances.append(1)
        for i in range(3):
            pull_known_lever(levers, output_sums, output_instances)
        current_lever += 1
    for i in range(100):
        pull_known_lever(levers, output_sums, output_instances)

This problem is well known in probability theory as the multi-armed bandit problem. However, most solutions assume four conditions that we are not given:

(1) The results from each lever are based on a continuous probability curve.

(2) The curve is without a significant skew.

(3) The number of arms is small.

(4) The amount of time it takes to test every arm is not worth considering, in comparison to the benefit of a preliminary examination of all variables.

These assumptions are useful, but not comprehensive. If I wish to address the problem I have been given, then solutions that make broad assumptions are undesirable. However, there is information that can be learned. A generic rule, Hoeffding's Inequality, can be used as a representation of all curves. It is a calculation for the maximum distance the true mean can vary from the real mean, within a certain confidence level, given the number of samples. By rearranging this equation, we can calculate a 95% confidence level around each lever. If the upper bound of this region is less than the lower bound of the best lever, than the current lever is no longer worth investigation. A solution to check the adjusted formula is included below:

In [0]:
def hoeffding_maximum_confident_displacement(number_of_samples, given_range):
    return 1.9206455826 * given_range / math.sqrt(number_of_samples)


def hoeffding_comparison(high_number_of_samples, high_mean, low_number_of_samples, low_mean, given_range):
    return low_mean + hoeffding_maximum_confident_displacement(low_number_of_samples, given_range) > high_mean - hoeffding_maximum_confident_displacement(high_number_of_samples, given_range)

An important standard used in the evaluation of solutions to the multi-armed bandit problem is cumulative regret. Cumulative regret is the difference between the estimated mean of the best lever and the estimated mean of each other lever, multiplied by the times the other lever is used.

However, it produces an incomplete evaluation. Consider an algorithm that picked one lever and pulled it indefinitely. It would never gain enough information to regret a decision. However, it would be unlikely to select the optimal lever. If every lever gets pulled only once, the regret increases, and the chance of settling upon the optimal lever increases. The challenge is to minimise regret and maximise chance of finding the ideal lever.

As a seperate factor, a balance should be made between long term and short term cumulative regret. For good long term results, the best test that can be done is trying each lever in order, until the lever is confirmed to be no longer useful. For better short term results, testing of levers unlikely to return high values should be delayed, or stopped entirely.

Now that the components have been isolated, it may be worth re-examining some variables. My earlier rate of testing 1 new lever every 4 pulls was based on the assumption that the array was not skewed. However, given that we are testing gambling machines, the existence of a right skew is likely, as the existence of high jackpots is often used. As a result, I will change the rate to 1 in 3 pulls. In addition, this may decrease the long term cumulative regret, as good levers later in the set can be found before a significant investment is made in other, earlier levers.

Another factor worth reevaluating is the scaling proportions of the best and worst levers. The existing practice of the algorithm, using squares to ensure that pulling of lesser levers drops off, has drawbacks. To minimise long-term regret, the algorithm would function best by pulling each lever in uniform proportions. To minimise short-term regret, the algorithm would function best by pulling the favored lever repeatedly. To compromise, and perform well uniformly, it should make many pulls of the best performing lever, but also pull other, milder performing levers. 

Factoring all of this into consideration, I have produced the following algorithm:

In [0]:
import math

def partition_levers(levers, output_sums, output_instances, low, high):
    i = (low - 1)
    pivot = output_sums[high] / output_instances[high]
    for j in range(low, high):
        if output_sums[j] / output_instances[j] >= pivot:
            i += 1
            levers[i], levers[j] = levers[j], levers[i]
            output_sums[i], output_sums[j] = output_sums[j], output_sums[i]
            output_instances[i], output_instances[j] = output_instances[j], output_instances[i]
    levers[i+1], levers[high] = levers[high], levers[i+1]
    output_sums[i+1], output_sums[high] = output_sums[high], output_sums[i+1]
    output_instances[i+1], output_instances[high] = output_instances[high], output_instances[i+1]
    return (i+1)


def quick_lever_sort(levers, output_sums, output_instances, low, high):
    if low < high:
        pi = partition_levers(levers, output_sums, output_instances, low, high)
        quick_lever_sort(levers, output_sums, output_instances, low, pi-1)
        quick_lever_sort(levers, output_sums, output_instances, pi+1, high)
        
        
def hoeffding_maximum_confident_displacement(number_of_samples, given_range):
    return 1.9206455826 * given_range / math.sqrt(number_of_samples)


def hoeffding_comparison(high_number_of_samples, high_mean, low_number_of_samples, low_mean, given_range):
    return low_mean + hoeffding_maximum_confident_displacement(low_number_of_samples, given_range) > high_mean - hoeffding_maximum_confident_displacement(high_number_of_samples, given_range)


def pull_known_lever(levers, output_sums, output_instances, given_range):
    quick_lever_sort(levers, output_sums, output_instances, 0, len(output_sums) - 1)
    known_levers = len(output_instances)
    minimum_lever_exponential_base = pow(output_instances[0], 1 / (2.0*known_levers))
    minimum_lever_exponential_base_index = 0
    for i in range(1, known_levers):
        if hoeffding_comparison(output_instances[0], output_sums[0] / output_instances[0], output_instances[i], output_sums[i] / output_instances[i], given_range):
            lever_exponential_base = pow(output_instances[i], 1 / (2.0*known_levers-i))
            if lever_exponential_base < minimum_lever_exponential_base:
                minimum_lever_exponential_base = lever_exponential_base
                minimum_lever_exponential_base_index = i
    result = pull_lever(levers[minimum_lever_exponential_base_index])
    output_sums[minimum_lever_exponential_base_index] += pull_lever(levers[minimum_lever_exponential_base_index])
    output_instances[minimum_lever_exponential_base_index] += 1
    return result


def algorithm(levers, pulls):
    output_sums = []
    output_instances = []
    minimum_value = pull_lever(levers[0])
    maximum_value = minimum_value
    intermediate_range = 0.001
    output_sums.append(minimum_value)
    output_instances.append(1)
    count = 1
    current_lever = 1
    while current_lever < len(levers):
        recent_result = pull_lever(levers[current_lever])
        count += 1
        if recent_result < minimum_value:
            minimum_value = recent_result
            intermediate_range = maximum_value - minimum_value
        elif recent_result > maximum_value:
            maximum_value = recent_result
            intermediate_range = maximum_value - minimum_value
        output_sums.append(recent_result)
        output_instances.append(1)
        for i in range(2):
            recent_result = pull_known_lever(levers, output_sums, output_instances, intermediate_range)
            count += 1
            if recent_result < minimum_value:
                minimum_value = recent_result
                intermediate_range = maximum_value - minimum_value
            elif recent_result > maximum_value:
                maximum_value = recent_result
                intermediate_range = maximum_value - minimum_value
        current_lever += 1
    for i in range(count, pulls):
        recent_result = pull_known_lever(levers, output_sums, output_instances, intermediate_range)
        if recent_result < minimum_value:
            minimum_value = recent_result
            intermediate_range = maximum_value - minimum_value
        elif recent_result > maximum_value:
            maximum_value = recent_result
            intermediate_range = maximum_value - minimum_value
    result_sum = 0
    for current_sum in output_sums:
        result_sum += current_sum
    return result_sum

For the purposes of testing, a comprehensive lever capable of replicating a variety of test conditions is desirable.

In [0]:
import random

def pull_lever(lever):
  if lever[0] == 0:
    if lever[1] < random.random():
      return 1
    else:
      return 0
  else:
    return 0

As an initial test, I will try using a series of 10 levers, each with a different chance of returning a consistent reward, from 0% to 90%.

In 128 pulls, a solution with perfect information would on average

In [43]:
# in order
levers = [[0,0.0],[0,0.1],[0,0.2],[0,0.3],[0,0.4],[0,0.5],[0,0.6],[0,0.7],[0,0.8],[0,0.9]]
print(algorithm(levers,128))

# in reverse order
levers = [[0,0.9],[0,0.8],[0,0.7],[0,0.6],[0,0.5],[0,0.4],[0,0.3],[0,0.2],[0,0.1],[0,0.0]]
print(algorithm(levers,128))

# in random orders
random.shuffle(levers)
print(algorithm(levers,128))

(83, 128)
(77, 128)
(83, 128)


Through repeated testing, I was able to accumulate the following averages, after 128 lever pulls:

in order: 83.08

in reverse order: 83.36

in random order: 85.76

In this, grouping similar levers makes it difficult for the program to identify the ideal solution.